In [ ]:
import os, json, shutil
from pathlib import Path

import torch
from torchvision import models
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

INPUT_DIR  = r"C:\Desktop\ML project\ai-image-detector-comparison\data\raw_images"
OUTPUT_DIR = r"C:\Desktop\ML project\ai-image-detector-comparison\data\sorted"
RESULTS_CSV = r"C:\Desktop\ML project\ai-image-detector-comparison\data\classifier_results.csv"

EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
TOP1_THRESHOLD = 0.25  

KEYWORDS = {
    "people": ["person","man","woman","boy","girl","bride","groom","diver","skier","player","guitarist","singer"],
    "bird":   ["bird","eagle","hawk","falcon","parrot","duck","goose","swan","owl","hen","rooster"],
    "animal": ["dog","cat","lion","tiger","leopard","bear","horse","cow","sheep","goat","pig","deer","monkey","rabbit","fox","wolf"],
    "vehicle":["car","taxi","bus","truck","motorcycle","bike","bicycle","train","airplane","helicopter","boat","ship","sail"],
    "indoor": ["sofa","couch","chair","table","desk","lamp","tv","monitor","keyboard","bed"],
    "outdoor":["mountain","beach","forest","valley","cliff","lakeside","seashore","desert"],
    "building":["castle","palace","mosque","church","pagoda","barn","bridge","lighthouse"],
}
DEFAULT_CLASS = "other"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

weights = models.ResNet50_Weights.IMAGENET1K_V2
model = models.resnet50(weights=weights).to(device).eval()
preprocess = weights.transforms()
classes = weights.meta["categories"]